In [1]:
import pandas as pd
import spacy

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [2]:
train_data = pd.read_csv("/content/drive/MyDrive/dataset/word2vec/dataset/treino.csv")
train_data.sample(5)

,title,text,date,category,subcategory,link
16975,Ao menos 170 torcedores são detidos em Minas a...,Ao menos 170 torcedores do Atlético-MG foram d...,2016-03-27,esporte,NaN,http://www1.folha.uol.com.br/esporte/2016/03/1...
34351,"Com queda da Selic, consumidor tem janela para...",O ano começou com um sinal de alívio para deve...,2017-01-16,mercado,NaN,http://www1.folha.uol.com.br/mercado/2017/01/1...
64592,Uau! Cunha retira FGTS na Suíça!,Buemba! Buemba! Macaco Simão Urgente! O esculh...,2017-03-15,colunas,josesimao,http://www1.folha.uol.com.br/colunas/josesimao...
39154,"Produção industrial cresce 2,3% em dezembro, m...",A indústria brasileira fechou 2016 com queda d...,2017-01-02,mercado,NaN,http://www1.folha.uol.com.br/mercado/2017/02/1...
49335,"\n\t\tEmpty, New Metro Line to Olympic Park Is...",ITALO NOGUEIRA FROM RIO The new metro line th...,2016-02-08,esporte,olimpiada-no-rio,http://www1.folha.uol.com.br/esporte/olimpiada...


In [ ]:
#execute just once
#!python -m spacy download pt_core_news_sm

In [6]:
nlp = spacy.load("pt_core_news_sm")

In [7]:
text = "Rio de Janeiro é uma cidade maravilhosa"
doc = nlp(text)
doc

Rio de Janeiro é uma cidade maravilhosa

In [8]:
type(doc[2])

spacy.tokens.token.Token

In [9]:
texts_to_treatment = (titles.lower() for titles in train_data["title"])

In [10]:
def treat_texts(doc):
    valid_tokens = []
    for token in doc:
        is_valid = not token.is_stop and token.is_alpha
        if is_valid:
            valid_tokens.append(token.text)

    if len(valid_tokens) > 2:
        return  " ".join(valid_tokens)

text = "Rio de Janeiro 1231231 ***** @#$ é uma cidade maravilhosa!"
doc = nlp(text)
treat_texts(doc)

'Rio Janeiro cidade maravilhosa'

In [11]:
text = "Rio de Janeiro 1231231 ***** @#$ é uma cidade maravilhosa!"
doc = nlp(text)
treat_texts(doc)

'Rio Janeiro cidade maravilhosa'

In [12]:
from time import time

t0 = time()
treated_texts = [treat_texts(doc) for doc in nlp.pipe(texts_to_treatment,
                                                        batch_size = 1000,
                                                        n_process = -1)]
tf = time() - t0
print(tf/60)

2.9332737843195598


In [13]:
treated_titles = pd.DataFrame({"titulo": treated_texts})
treated_titles.head()

,titulo
0,polêmica marine le pen abomina negacionistas h...
1,macron le pen turno frança revés siglas tradic...
2,apesar larga vitória legislativas macron terá ...
3,governo antecipa balanço alckmin anuncia queda...
4,queda maio atividade econômica sobe junho bc


In [14]:
from gensim.models import Word2Vec

w2v_modelo = Word2Vec(sg = 0,
                      window = 2,
                      size = 300,
                      min_count = 5,
                      alpha = 0.03,
                      min_alpha = 0.007)

In [15]:
print(len(treated_titles))
treated_titles = treated_titles.dropna().drop_duplicates()
print(len(treated_titles))

90000
84466


In [16]:
list_tokens_list = [title.split(" ") for title in treated_titles.titulo]

In [17]:
import logging

logging.basicConfig(format="%(asctime)s : - %(message)s", level = logging.INFO)

w2v_model = Word2Vec(sg = 0,
                      window = 2,
                      size = 300,
                      min_count = 5,
                      alpha = 0.03,
                      min_alpha = 0.007)

w2v_model.build_vocab(list_tokens_list, progress_per=5000)

In [18]:
dir(w2v_model)

['__class__',
 '__contains__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_adapt_by_suffix',
 '_check_input_data_sanity',
 '_check_training_sanity',
 '_clear_post_train',
 '_do_train_epoch',
 '_do_train_job',
 '_get_job_params',
 '_get_thread_working_mem',
 '_job_producer',
 '_load_specials',
 '_log_epoch_end',
 '_log_epoch_progress',
 '_log_progress',
 '_log_train_end',
 '_minimize_model',
 '_raw_word_count',
 '_save_specials',
 '_set_train_params',
 '_smart_save',
 '_train_epoch',
 '_train_epoch_corpusfile',
 '_update_job_params',
 '_worker_loop',
 '_worker_loop_corpusfile',
 'accuracy',
 'alpha',
 'batch_words',
 'build_vocab',
 'build_vocab_from_freq',
 'ca

In [19]:
w2v_model.corpus_count

84466

In [20]:
w2v_model.train(list_tokens_list, 
                 total_examples=w2v_model.corpus_count,
                 epochs = 30)

(14584596, 16207260)

In [21]:
w2v_model.wv.most_similar("google")

[('apple', 0.5938615798950195),
 ('facebook', 0.531114935874939),
 ('amazon', 0.5061439275741577),
 ('news', 0.4816396236419678),
 ('fbi', 0.4781722128391266),
 ('uber', 0.477141797542572),
 ('airbnb', 0.4767501950263977),
 ('waze', 0.47197070717811584),
 ('volkswagen', 0.46985331177711487),
 ('sony', 0.4559990167617798)]

In [22]:
w2v_model.wv.most_similar("microsoft")

[('amazon', 0.5898116827011108),
 ('unilever', 0.5881766080856323),
 ('viajante', 0.5370166301727295),
 ('inbev', 0.5238409638404846),
 ('sony', 0.5219316482543945),
 ('tesla', 0.5216419100761414),
 ('spotify', 0.5174892544746399),
 ('walmart', 0.5171464681625366),
 ('sky', 0.5166171789169312),
 ('chrysler', 0.5073357820510864)]

In [23]:
w2v_model.wv.most_similar("barcelona")

[('barça', 0.617017388343811),
 ('madrid', 0.5761963129043579),
 ('bayern', 0.5746067762374878),
 ('psg', 0.5718961358070374),
 ('munique', 0.5504690408706665),
 ('chelsea', 0.5415440797805786),
 ('atlético', 0.5382505059242249),
 ('juventus', 0.5212011933326721),
 ('coritiba', 0.5209093689918518),
 ('tottenham', 0.5154260396957397)]

In [24]:
w2v_model.wv.most_similar("messi")

[('suárez', 0.5786398649215698),
 ('cavani', 0.5278909802436829),
 ('cristiano', 0.5252474546432495),
 ('neymar', 0.5126093626022339),
 ('benzema', 0.506260871887207),
 ('barcelona', 0.49391356110572815),
 ('chuteiras', 0.493035227060318),
 ('barça', 0.49128854274749756),
 ('mordida', 0.48801228404045105),
 ('psg', 0.48611316084861755)]

In [25]:
w2v_model.wv.most_similar("gm")

[('embraer', 0.6954017877578735),
 ('volks', 0.676307201385498),
 ('chrysler', 0.6711773872375488),
 ('honda', 0.6514461040496826),
 ('braskem', 0.6290566921234131),
 ('toyota', 0.6266669034957886),
 ('tesla', 0.6099504232406616),
 ('volkswagen', 0.6091933250427246),
 ('fiat', 0.6026105880737305),
 ('renault', 0.5737369060516357)]

In [26]:
#model Skip-Gram training
w2v_model_sg = Word2Vec(sg = 1,
                      window = 5,
                      size = 300,
                      min_count = 5,
                      alpha = 0.03,
                      min_alpha = 0.007)

w2v_model_sg.build_vocab(list_tokens_list, progress_per=5000)

w2v_model_sg.train(list_tokens_list, 
                 total_examples=w2v_model_sg.corpus_count,
                 epochs = 30)

(14584461, 16207260)

In [27]:
w2v_model_sg.wv.most_similar("google")

[('android', 0.40671637654304504),
 ('patentes', 0.39327797293663025),
 ('buffett', 0.38503605127334595),
 ('apple', 0.373024046421051),
 ('waze', 0.3691113591194153),
 ('reguladores', 0.3677915334701538),
 ('anunciantes', 0.3650829493999481),
 ('concorda', 0.3582262396812439),
 ('facebook', 0.35227471590042114),
 ('app', 0.350563108921051)]

In [28]:
w2v_model.wv.most_similar("google")

[('apple', 0.5938615798950195),
 ('facebook', 0.531114935874939),
 ('amazon', 0.5061439275741577),
 ('news', 0.4816396236419678),
 ('fbi', 0.4781722128391266),
 ('uber', 0.477141797542572),
 ('airbnb', 0.4767501950263977),
 ('waze', 0.47197070717811584),
 ('volkswagen', 0.46985331177711487),
 ('sony', 0.4559990167617798)]

In [29]:
w2v_model_sg.wv.most_similar("gm")

[('metalúrgicos', 0.5697438716888428),
 ('motors', 0.5151419639587402),
 ('audi', 0.5023962259292603),
 ('honda', 0.4984953701496124),
 ('cubatão', 0.48548832535743713),
 ('bmw', 0.47998031973838806),
 ('chrysler', 0.4703269600868225),
 ('airbag', 0.4696553647518158),
 ('airbags', 0.46397942304611206),
 ('dupont', 0.46296611428260803)]

In [30]:
w2v_model.wv.most_similar("gm")

[('embraer', 0.6954017877578735),
 ('volks', 0.676307201385498),
 ('chrysler', 0.6711773872375488),
 ('honda', 0.6514461040496826),
 ('braskem', 0.6290566921234131),
 ('toyota', 0.6266669034957886),
 ('tesla', 0.6099504232406616),
 ('volkswagen', 0.6091933250427246),
 ('fiat', 0.6026105880737305),
 ('renault', 0.5737369060516357)]

In [33]:
w2v_model.wv.save_word2vec_format("/content/drive/MyDrive/dataset/word2vec/dataset/modelo_cbow.txt", binary=False)
w2v_model_sg.wv.save_word2vec_format("/content/drive/MyDrive/dataset/word2vec/dataset/modelo_skipgram.txt", binary=False)
train_article = pd.read_csv("/content/drive/MyDrive/dataset/word2vec/dataset/treino.csv")
test_article = pd.read_csv("/content/drive/MyDrive/dataset/word2vec/dataset/teste.csv")

In [34]:
nlp = spacy.load("pt_core_news_sm", disable=["paser", "ner", "tagger", "textcat"])

def tokenizer(text):
    
    doc = nlp(text)
    valid_tokens = []
    for token in doc:
        is_valid = not token.is_stop and token.is_alpha
        if is_valid:
            valid_tokens.append(token.text.lower())

    
    return valid_tokens

text = "Rio de Janeiro 1231231 ***** @#$ é uma cidade maravilhosa!"
tokens = tokenizer(text)
print(tokens)

['rio', 'janeiro', 'cidade', 'maravilhosa']


In [37]:
import numpy as np

def combination_of_vectors_by_sum(words, model):

    result_vector = np.zeros((1,300))

    for wd in words:
        try:
            result_vector += model.get_vector(wd)

        except KeyError:
            pass
                

    return result_vector

text_vector = combination_of_vectors_by_sum(tokens, w2v_model.wv)
print(text_vector.shape)
print(text_vector)

(1, 300)
[[ 0.41325611 -1.44723862 -1.03914675 -0.86961566 -1.1723512   0.25786437
   1.60502013  0.99500862 -0.31147689  0.0198146   1.25584805 -0.21678257
  -2.15993881 -1.21405482  1.50712779 -0.03214464  0.7728097  -1.28045946
   0.14574212 -2.19041215  3.31831983 -1.29058843 -0.35271397  0.80130688
  -1.24308884  0.41417759 -1.44460563 -1.02658887  0.83419821  2.11004092
  -0.13568562 -0.80210886 -2.87711585  1.96403018  0.30374343 -0.97556084
   0.13416898  1.13175738  0.38567679 -1.0381768   1.8429294   1.76735888
  -0.65771142 -1.24211502  3.55596954  1.42196102  1.09856462 -0.77323397
  -1.56972072  1.99855489 -0.1281274   0.22341104 -1.77500159  0.39680309
  -2.15866688  0.12803494 -1.00443116  0.12109391 -0.77953321  0.4446521
  -0.49572591 -0.66148297  0.23600222  2.92412502 -1.85312681  0.13629064
  -1.61016561  0.26852022  0.13933636  2.06354755  0.74911889 -0.66636961
  -1.206587    0.2531739  -1.51172397 -0.4144102   0.35139532  0.48165165
  -1.23780161 -0.13225311  1.0

In [39]:
def matrix_vectors(texts, model):
    x = len(texts)
    y = 300
    matrix = np.zeros((x,y))

    for i in range(x):
        words = tokenizer(texts.iloc[i])
        matrix[i] = combination_of_vectors_by_sum(words, model)

    return matrix

matrix_vectors_train_cbow = matrix_vectors(train_article.title, w2v_model.wv)
matrix_vectors_test_cbow = matrix_vectors(test_article.title, w2v_model.wv)
print(matrix_vectors_train_cbow.shape)
print(matrix_vectors_test_cbow.shape) 

(90000, 300)
(20513, 300)
